In [1]:
import requests
import bs4
import re
import csv
from string import ascii_lowercase
import os

In [11]:
def create_csv_file():
    if 'ufc_fighter_data.csv' not in os.listdir():
        with open('ufc_fighter_data.csv','w',newline='',encoding='UTF8') as ufc_fighter_data:
            writer = csv.writer(ufc_fighter_data)
            writer.writerow(['fighter_f_name',
                             'fighter_l_name',
                             'fighter_nickname',
                             'fighter_height_cm',
                             'fighter_weight_lbs',
                             'fighter_reach_cm',
                             'fighter_stance',
                             'fighter_dob',
                             'fighter_w',
                             'fighter_l',
                             'fighter_d',
                             'fighter_nc_dq',
                             'fighter_url'])
    else:
        pass

In [4]:
def get_fighter_urls():
    main_url_list = [requests.get(f'http://ufcstats.com/statistics/fighters?char={letter}&page=all') for letter in ascii_lowercase]
    main_soup_list = [bs4.BeautifulSoup(url.text,'lxml') for url in main_url_list]
    all_links = []
    
    for item in main_soup_list:
        for item in item.select('a.b-link')[1::3]:
            all_links.append(item.get('href')) 
    
    return all_links

In [36]:
def get_fighter_data(fighter_urls):
    
    with open('ufc_fighter_data.csv','a+') as ufc_fighters:
        writer = csv.writer(ufc_fighters)
        
    
        for url in fighter_urls:
            fighter_url = requests.get(url)
            fighter_soup = bs4.BeautifulSoup(fighter_url.text,'lxml')

            select_name = fighter_soup.select('span')[0].text.split()
            select_nickname = fighter_soup.select('p.b-content__Nickname')
            select_details = fighter_soup.select('li.b-list__box-list-item')
            select_record = fighter_soup.select('span.b-content__title-record')[0].text.split(':')[1].strip().split('-')

            if len(select_name)>1:
                fighter_f_name = select_name[0]
                fighter_l_name = select_name[-1]
            else:
                fighter_f_name = select_name[0]
                fighter_l_name = None
            if select_nickname[0].text == '\n':
                fighter_nickname = None
            else:
                fighter_nickname = select_nickname[0].text
            if '--' in select_details[0].text.split(':')[1].strip().split("'"):
                fighter_height_cm = None
            else:
                fighter_height_ft = select_details[0].text.split(':')[1].strip()[0]
                fighter_height_in = select_details[0].text.split(':')[1].strip().split("'")[1].strip().strip('"')
                fighter_height_cm = ((int(fighter_height_ft)*12.0)*2.54)+(int(fighter_height_in)*2.54)
            if '--' in select_details[1].text.split(':')[1]:
                fighter_weight_lbs = None
            else:
                fighter_weight_lbs = select_details[1].text.split(':')[1].split()[0]
            if '--' in select_details[2].text.split(':')[1]:
                fighter_reach_cm = None
            else:
                fighter_reach_cm = round(int(select_details[2].text.split(':')[1].strip().strip('"'))*2.54,2)
            if select_details[3].text.split(':')[1].strip()=='':
                fighter_stance = None
            else:
                fighter_stance = select_details[3].text.split(':')[1].strip()
            fighter_dob = select_details[4].text.split(':')[1]
            fighter_w = select_record[0]
            fighter_l = select_record[1]
            if len(select_record[-1]) > 1:
                fighter_d = select_record[-1][0]
            else:
                fighter_d = select_record[-1]
            if len(select_record[-1]) > 1:
                 fighter_nc_dq = select_record[-1].split('(')[-1][0]
            else:
                fighter_nc_dq = None

            writer.writerow([fighter_f_name.strip(), 
                             fighter_l_name.strip(), 
                             fighter_nickname.strip(),
                             fighter_height_cm,
                             fighter_weight_lbs.strip(),
                             fighter_reach_cm,
                             fighter_stance.strip(),
                             fighter_dob.strip(),
                             fighter_w,
                             fighter_l,
                             fighter_d,
                             fighter_nc_dq,
                             url])

In [34]:
create_csv_file()

In [35]:
get_fighter_data(fighter_urls)